In [1]:
pip install pyspark

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 14.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=74c57ef238bf223bbaff73f4c300cefe17d54234ffb6f4e183a83d7b61ae62ea
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('DfPractice').getOrCreate()

In [5]:
spark

In [6]:
spark.read.csv("/content/test2.csv")

DataFrame[_c0: string, _c1: string, _c2: string]

In [8]:
df_spark = spark.read.option('header','true').csv('test2.csv')

In [9]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Experience: string (nullable = true)



In [10]:
df_spark = spark.read.option('header','true').csv('test2.csv', inferSchema = True)

In [11]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [12]:
df_spark = spark.read.csv('test2.csv', header = True, inferSchema = True)

In [13]:
df_spark.show()

+-----+---+----------+
| Name|Age|Experience|
+-----+---+----------+
| Abba| 31|         3|
|Jabba| 21|         2|
|Dabba| 23|         1|
+-----+---+----------+



In [14]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [15]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [16]:
df_spark.columns

['Name', 'Age', 'Experience']

In [17]:
df_spark.select('Name').show()

+-----+
| Name|
+-----+
| Abba|
|Jabba|
|Dabba|
+-----+



In [18]:
df_spark.select(['Name','Experience']).show()

+-----+----------+
| Name|Experience|
+-----+----------+
| Abba|         3|
|Jabba|         2|
|Dabba|         1|
+-----+----------+



In [19]:
df_spark['Name']

Column<'Name'>

In [20]:
df_spark.dtypes

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int')]

In [22]:
df_spark.describe().show()

+-------+-----+-----------------+----------+
|summary| Name|              Age|Experience|
+-------+-----+-----------------+----------+
|  count|    3|                3|         3|
|   mean| null|             25.0|       2.0|
| stddev| null|5.291502622129181|       1.0|
|    min| Abba|               21|         1|
|    max|Jabba|               31|         3|
+-------+-----+-----------------+----------+



In [23]:
#adding column
df_spark.withColumn('Experience after 2 years',df_spark['Experience']+2).show()

+-----+---+----------+------------------------+
| Name|Age|Experience|Experience after 2 years|
+-----+---+----------+------------------------+
| Abba| 31|         3|                       5|
|Jabba| 21|         2|                       4|
|Dabba| 23|         1|                       3|
+-----+---+----------+------------------------+



In [24]:
 df_spark.drop('Experience after 2 years')

DataFrame[Name: string, Age: int, Experience: int]

In [25]:
df_spark.show()

+-----+---+----------+
| Name|Age|Experience|
+-----+---+----------+
| Abba| 31|         3|
|Jabba| 21|         2|
|Dabba| 23|         1|
+-----+---+----------+



In [26]:
df_spark.withColumnRenamed('Name','EmpName').show()

+-------+---+----------+
|EmpName|Age|Experience|
+-------+---+----------+
|   Abba| 31|         3|
|  Jabba| 21|         2|
|  Dabba| 23|         1|
+-------+---+----------+



In [31]:
spark.read.csv("/content/test3.csv")

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]

In [32]:
df_pyspark = spark.read.csv('test3.csv', header = True, inferSchema = True)

In [34]:
df_pyspark.show()

+-----+----+----------+-------+
| Name| Age|Experience| Salary|
+-----+----+----------+-------+
| Abba|  31|         3|  30000|
|Jabba|  21|         2|  40000|
|Dabba|  23|         1|  20000|
| cat |null|         8|  73637|
|  dog|  21|      null|   null|
|  stu|  22|      null|   2364|
| Stut|  23|        23|2324422|
+-----+----+----------+-------+



In [35]:
df_pyspark.drop('Name').show()

+----+----------+-------+
| Age|Experience| Salary|
+----+----------+-------+
|  31|         3|  30000|
|  21|         2|  40000|
|  23|         1|  20000|
|null|         8|  73637|
|  21|      null|   null|
|  22|      null|   2364|
|  23|        23|2324422|
+----+----------+-------+



In [37]:
df_pyspark.na.drop().show()

+-----+---+----------+-------+
| Name|Age|Experience| Salary|
+-----+---+----------+-------+
| Abba| 31|         3|  30000|
|Jabba| 21|         2|  40000|
|Dabba| 23|         1|  20000|
| Stut| 23|        23|2324422|
+-----+---+----------+-------+



In [60]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age','Experience'],
    outputCols=["{}_imputed".format(c) for c in ['Age','Experience']]
    ).setStrategy("mean")

In [62]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+----+----------+-------+-----------+------------------+
| Name| Age|Experience| Salary|Age_imputed|Experience_imputed|
+-----+----+----------+-------+-----------+------------------+
| Abba|  31|         3|  30000|         31|                 3|
|Jabba|  21|         2|  40000|         21|                 2|
|Dabba|  23|         1|  20000|         23|                 1|
| cat |null|         8|  73637|         23|                 8|
|  dog|  21|      null|   null|         21|                 7|
|  stu|  22|      null|   2364|         22|                 7|
| Stut|  23|        23|2324422|         23|                23|
+-----+----+----------+-------+-----------+------------------+



In [63]:
df_pyspark.filter('Salary<=20000').show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Dabba| 23|         1| 20000|
|  stu| 22|      null|  2364|
+-----+---+----------+------+



In [64]:
df_pyspark.filter('Salary<=20000').select(['Name','Age']).show()

+-----+---+
| Name|Age|
+-----+---+
|Dabba| 23|
|  stu| 22|
+-----+---+



In [65]:
df_pyspark.filter(df_pyspark['Salary']<=20000).show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Dabba| 23|         1| 20000|
|  stu| 22|      null|  2364|
+-----+---+----------+------+



In [70]:
df_pyspark.filter((df_pyspark['Salary']<=20000) & (df_pyspark['Age']==23)).show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Dabba| 23|         1| 20000|
+-----+---+----------+------+



In [71]:
df_pyspark.filter(~(df_pyspark['Salary']<=20000) & (df_pyspark['Age']==23)).show()

+----+---+----------+-------+
|Name|Age|Experience| Salary|
+----+---+----------+-------+
|Stut| 23|        23|2324422|
+----+---+----------+-------+



In [74]:
df_pyspark.groupby('Name').sum().show()

+-----+--------+---------------+
| Name|sum(Age)|sum(Experience)|
+-----+--------+---------------+
| Stut|      23|             23|
|  dog|      21|           null|
| cat |    null|              8|
| Abba|      31|              3|
|Jabba|      21|              2|
|  stu|      22|           null|
|Dabba|      23|              1|
+-----+--------+---------------+



In [75]:
df_pyspark.groupby('Age').mean().show()

+----+--------+---------------+
| Age|avg(Age)|avg(Experience)|
+----+--------+---------------+
|  31|    31.0|            3.0|
|  22|    22.0|           null|
|null|    null|            8.0|
|  23|    23.0|           12.0|
|  21|    21.0|            2.0|
+----+--------+---------------+

